In [ ]:
from tifffile import imread, imsave
import os, re, sys, csv
import numpy as np
import matplotlib.pyplot as pyp
from skimage.morphology import remove_small_objects, binary_closing
from skimage.segmentation import find_boundaries
from skimage.measure import regionprops, regionprops_table, label
from skimage.segmentation import clear_border
import cv2
import copy
import pandas as pd
from scipy import ndimage as ndi
import napari
import scanpy as sc
import seaborn as sns
from skimage.future.graph import RAG, rag_mean_color
import math

In [ ]:
## function from skimage package https://github.com/scikit-image/scikit-image/blob/main/skimage/measure/_regionprops.py#L869-L1161

COL_DTYPES = {
    'area': int,
    'bbox': int,
    'bbox_area': int,
    'moments_central': float,
    'centroid': float,
    'convex_area': int,
    'convex_image': object,
    'coords': object,
    'eccentricity': float,
    'equivalent_diameter': float,
    'euler_number': int,
    'extent': float,
    'feret_diameter_max': float,
    'filled_area': int,
    'filled_image': object,
    'moments_hu': float,
    'image': object,
    'inertia_tensor': float,
    'inertia_tensor_eigvals': float,
    'intensity_image': object,
    'label': int,
    'local_centroid': float,
    'major_axis_length': float,
    'max_intensity': int,
    'mean_intensity': float,
    'min_intensity': int,
    'minor_axis_length': float,
    'moments': float,
    'moments_normalized': float,
    'orientation': float,
    'perimeter': float,
    'slice': object,
    'solidity': float,
    'weighted_moments_central': float,
    'weighted_centroid': float,
    'weighted_moments_hu': float,
    'weighted_local_centroid': float,
    'weighted_moments': float,
    'weighted_moments_normalized': float
}

OBJECT_COLUMNS = {
    'image', 'coords', 'convex_image', 'slice',
    'filled_image', 'intensity_image'
}

def  skimage_props_to_dict(regions, properties=('label', 'bbox'), separator='-'):
    """Convert image region properties list into a column dictionary."""

    out = {}
    n = len(regions)
    for prop in properties:
        r = regions[0]
        rp = getattr(r, prop)
        if prop in COL_DTYPES:
            dtype = COL_DTYPES[prop]
        else:
            func = r._extra_properties[prop]
            dtype = _infer_regionprop_dtype(
                func,
                intensity=r._intensity_image is not None,
                ndim=r.image.ndim,
            )
        column_buffer = np.zeros(n, dtype=dtype)

        # scalars and objects are dedicated one column per prop
        # array properties are raveled into multiple columns
        # for more info, refer to notes 1
        if np.isscalar(rp) or prop in OBJECT_COLUMNS or dtype is np.object_:
            for i in range(n):
                column_buffer[i] = regions[i][prop]
            out[prop] = np.copy(column_buffer)
        else:
            if isinstance(rp, np.ndarray):
                shape = rp.shape
            else:
                shape = (len(rp),)

            for ind in np.ndindex(shape):
                for k in range(n):
                    loc = ind if len(ind) > 1 else ind[0]
                    column_buffer[k] = regions[k][prop][loc]
                modified_prop = separator.join(map(str, (prop,) + ind))
                out[modified_prop] = np.copy(column_buffer)
    return out

### Set inputs

In [ ]:
# INPUT: single chanel TIFFs from the whole 3D model to use for 

#folder for registeration i.e an image per slice
input_base = '~/3D_model201710/3D_registred_tiffs/IMC_fullStack_registred/imageJ_registration/full_model_aligned/'
vessel_mask_input = input_base + 'blood_vessel_mask.tif'
cell_labels_input = input_base + "measured_mask_final_segmentation_hwatershed_500.00_90%.tif"

results_file = input_base +'model201710_singleCell_analysis.h5ad'  # the file that will store the analysis results


In [ ]:
vessel = imread(input_base + "CD31vWF_ROI_image.tif")

In [ ]:
adata = sc.read_h5ad(results_file)
o = list(adata.obs['phenograph'])
c = list(adata.obs['cell_labels'])

cluster_cell_label_dictionary = {}

for item in range(len(o)):
    dict_key = int(c[item])
    cluster_cell_label_dictionary[dict_key]= o[item]

In [ ]:
cell_labels = imread(cell_labels_input)

#### Calculate nieghbourhood graph.
Use vessel ROI image to have weights for the RAG but weights are not used

In [ ]:
rag_for_cell_labels =  rag_mean_color(vessel,cell_labels, connectivity=2)

In [ ]:
cell_labels_with_subset = cell_labels[32, :, :]
cells_on_slice = np.unique(cell_labels_with_subset) 

In [ ]:
nbr_cluster_dict = {}
for cell in c:
    if cell > 0:
        all_nbrs = rag_for_cell_labels.adj[cell]
        nbr_cluster_dict[cell] = []
        for entry in all_nbrs.keys():
            if entry > 0:
                nbr_cluster_dict[cell].append(cluster_cell_label_dictionary[entry])

In [ ]:
type_dict = {}
for e in nbr_cluster_dict:
    nbrs_types = nbr_cluster_dict[e]
    total_nbrs = len(nbrs_types)
    type_dict[e] = {}
    for i in range(38):
        if total_nbrs == 0:
            type_dict[e][i] = 0
        else:
            amount = nbrs_types.count(i) #/total_nbrs
            type_dict[e][i] = amount

In [ ]:
nbrs_table_3D = pd.DataFrame.from_dict(data = type_dict,orient = 'index')
table_name = input_base + "total_counts_cell_neigbor_identities_3D_all_cells.csv"
nbrs_table_3D.to_csv(table_name)

#### Read in adjacent neigbor labels tabel for 3D computed from RAG

In [ ]:
nbrs_table_3D = pd.read_csv(input_base + "total_counts_cell_neigbor_identities_3D_all_cells.csv", index_col = 0)
nbrs_table_3D = nbrs_table_3D.drop('0',1)

In [ ]:
cluster_labels = list(adata.obs['phenograph'])
object_labels = list(adata.obs['cell_labels'])

cluster_label_dictionary = {}
for item in range(len(object_labels)):
    dict_key = int(cluster_labels[item])
    if dict_key in cluster_label_dictionary.keys():
        cluster_label_dictionary[dict_key].append(int(object_labels[item]))
        
    else:    
        cluster_label_dictionary[dict_key] = []
        cluster_label_dictionary[dict_key].append(int(object_labels[item]))

In [ ]:
t_cell_labels = cluster_label_dictionary[1]
subset_table = nbrs_table_3D.filter(items = t_cell_labels, axis = 'index')
mean_nbrs = pd.DataFrame(subset_table.sum(axis=0))
mean_nbrs = mean_nbrs.rename(columns={0: 1})

for i in range(len(cluster_label_dictionary.keys())):
    if i == 1 or i ==0 :
        continue
    else:    
        cells = cluster_label_dictionary[i]
        subset_table = nbrs_table_3D.filter(items = cells, axis = 'index')
        means = pd.DataFrame(subset_table.sum(axis=0))
        mean_nbrs.insert(i-1, i, means)

In [ ]:
average_table = mean_nbrs
average_table['Total_3D']= average_table.sum(axis=1)
nbrs_3D_norm = (average_table.iloc[:][:].div(average_table['Total_3D'], axis = 0))*100
nbrs_3D_norm = nbrs_3D_norm.drop('Total_3D',1)

In [ ]:
nbrs_3D_norm

In [ ]:
ax = sns.clustermap(nbrs_3D_norm, row_cluster = False, col_cluster = False)

### Calculate for 2D slices

In [ ]:
nbrs_2d_dict = np.zeros((152,37,38), dtype = int)

for s in range(cell_labels.shape[0]):
            type_dict = {}

            cell_labels_slice = cell_labels[s, :, :]
            vessel_slice = vessel[s, :, :]

            rag_for_cell_labels =  rag_mean_color(vessel_slice,cell_labels_slice, connectivity=2)
            cell_labels_with_subset = cell_labels[s, :, :]
            cells_on_slice = np.unique(cell_labels_with_subset)
            nbr_cluster_dict = {}

            for cell in cells_on_slice:
                if cell > 0:
                    all_nbrs = rag_for_cell_labels.adj[cell]
                    nbr_cluster_dict[cell] = []
                    for entry in all_nbrs.keys():
                        if entry > 0:
                            nbr_cluster_dict[cell].append(cluster_cell_label_dictionary[entry])

            for e in nbr_cluster_dict:
                nbrs_types = nbr_cluster_dict[e]
                total_nbrs = len(nbrs_types)
                if e not in type_dict.keys():
                    type_dict[e] = {}
                    for i in range(38):
                        if total_nbrs == 0:
                            type_dict[e][i] = 0
                        else:
                            amount = nbrs_types.count(i)
                            type_dict[e][i] = amount                
                else:
                     for i in range(38):
                        if total_nbrs == 0:
                            continue
                        else:
                            amount = nbrs_types.count(i)
                            type_dict[e][i]= amount


            nbrs_table_2D = pd.DataFrame.from_dict(data = type_dict,orient = 'index')

            t_cell_labels = cluster_label_dictionary[1]
            subset_table = nbrs_table_2D.filter(items = t_cell_labels, axis = 'index')
            mean_nbrs = pd.DataFrame(subset_table.sum(axis=0))
            mean_nbrs = mean_nbrs.rename(columns={0: 1})

            for i in range(len(cluster_label_dictionary.keys())):
                if i == 1 or i ==0 :
                    continue
                else:    
                    cells = cluster_label_dictionary[i]
                    subset_table = nbrs_table_2D.filter(items = cells, axis = 'index')
                    means = pd.DataFrame(subset_table.sum(axis=0))
                    mean_nbrs.insert(i-1, i, means)
            mean_nbrs = mean_nbrs.drop(0,0)        
            mean_nbrs['Total_2D']= mean_nbrs.sum(axis=1)
            nbrs_2d_dict[s, :, :] = mean_nbrs       

In [ ]:
x = np.sum(nbrs_2d_dict, axis = 0)
totals_2d_model = x[:,37]
x = x[:, :-1]
avaerage_2d_proportions = x/totals_2d_model[:, None]
avaerage_2d_proportions = avaerage_2d_proportions*100
avaerage_2d_proportions= pd.DataFrame(avaerage_2d_proportions)

In [ ]:
ax = sns.clustermap(avaerage_2d_proportions, row_cluster = False, col_cluster = False)

In [ ]:
new_names = [i+1 for i in list(avaerage_2d_proportions.columns.values)]
avaerage_2d_proportions.columns = new_names
new_names = [i+1 for i in list(avaerage_2d_proportions.index.values)]
avaerage_2d_proportions.index = [str(i) for i in new_names]

In [ ]:
avaerage_2d_proportions.index.values

In [ ]:
divisionResults = nbrs_3D_norm.sub(avaerage_2d_proportions)
ax = sns.clustermap(divisionResults, row_cluster = False, col_cluster = False, cmap="RdBu" , vmin=-10, vmax=10)

###### End of notebook